## Flow cytometry simulation. 
Using ``NBNode`` as basis enables the simulation of (flow) cytometry data. Cytometry data consists of an unordered matrix of n ``cells`` and p ``features``. The ordering of cells in the matrix does not hold any significant information, except in cases where data corruption has occurred during the experiment.

As an example, we previously utilized a cell matrix originating from the Beckman Coulter ``DURAClone IM T Cell Subsets Antibody Panel``. This particular panel measures 10 markers in fluorochrome combinations that provide robust population identification, including CD3, CD4, CD8, CD27, CD28, CD45, CD45RA, CD57, CD197 (CCR7), CD279 (PD-1). Additionally, forward scatter and side scatter are usually measured, increasing the number of ``features`` per cell to 13. 

According to their ``feature`` values, ``cells`` can be classified ("gated") into cell types, e.g. ``CD4+ central memory T cells`` or ``CD27+ CD28+ CD4+ T cells``. Within one cell type, ``features`` do still vary. In this package, we use ``cell population`` for any defined set of cells. 

Our class ``FlowSimulationTree`` uses an existing gating applied to multiple samples and simulates two major parts: 

 1. How many ``cells`` are of which ``cell population``
 2. How are cell ``features`` distributed _within_ one ``cell population``
 
In this example, we will use data from 48 healthy human peripheral blood samples stained with the DURAClone IM T Cell Subsets Tube (Beckman Coulter GmbH). 
The data is available under [zenodo 7883353](https://doi.org/10.5281/zenodo.7883353). 


### Synthetic data based on previous samples

To enable the analysis also with data supplied with package, I supply the final result ``flowsim_tree.pickle`` which has done downloading gating and initiating the simulation. 

First, download the data. 

In [26]:
%%script false --no-raise-error

# This downloading takes about 1.5 minutes

# From https://github.com/zenodo/zenodo/issues/1888
import os

import requests

# params = {"access_token": "ZENODO_ACCESS_TOKEN"}  # only necessary until public
params = {}

record_id = "7883353"

r = requests.get(
    f"https://zenodo.org/api/records/{record_id}", params=params
)
print(r.json())
download_urls = [f["links"]["self"] for f in r.json()["files"]]
filenames = [f["key"] for f in r.json()["files"]]

print(r.status_code)
print(download_urls)

outdir = "example_data/asinh.align_manual.CD3_Gate"
os.makedirs(outdir, exist_ok=True)

for filename, url in zip(filenames, download_urls):
    print("Downloading:", filename)
    r = requests.get(url, params=params)
    with open(os.path.join(outdir, filename), "wb") as f:
        f.write(r.content)


The (pre-defined) tree for this dataset is ``nbtree.tree_complete_aligned_v2()``

In [4]:
%%script false --no-raise-error

import nbnode.nbnode_trees as nbtree
complete_tree = nbtree.tree_complete_aligned_v2()
complete_tree.pretty_print()

AllCells (counter:0)
├── DN (counter:0)
├── DP (counter:0)
├── CD4-/CD8+ (counter:0)
│   ├── naive (counter:0)
│   │   ├── CD27+/CD28+ (counter:0)
│   │   │   ├── CD57+/PD1+ (counter:0)
│   │   │   ├── CD57+/PD1- (counter:0)
│   │   │   └── CD57- (counter:0)
│   │   ├── CD27+/CD28- (counter:0)
│   │   │   ├── CD57+/PD1+ (counter:0)
│   │   │   ├── CD57+/PD1- (counter:0)
│   │   │   └── CD57- (counter:0)
│   │   ├── CD27-/CD28+ (counter:0)
│   │   │   ├── CD57+/PD1+ (counter:0)
│   │   │   ├── CD57+/PD1- (counter:0)
│   │   │   └── CD57- (counter:0)
│   │   └── CD27-/CD28- (counter:0)
│   │       ├── CD57+/PD1+ (counter:0)
│   │       ├── CD57+/PD1- (counter:0)
│   │       └── CD57- (counter:0)
│   ├── Tcm (counter:0)
│   │   ├── CD27+/CD28+ (counter:0)
│   │   │   ├── CD57+/PD1+ (counter:0)
│   │   │   ├── CD57+/PD1- (counter:0)
│   │   │   └── CD57- (counter:0)
│   │   ├── CD27+/CD28- (counter:0)
│   │   │   ├── CD57+/PD1+ (counter:0)
│   │   │   ├── CD57+/PD1- (counter:0)
│   │   │  

``nbtree.tree_complete_aligned_v2`` has specific ideas about the naming of 
the markers, so we need to use the same names here.

``new_colnames`` replaces the column names in all CSV files, regardless of how 
they were named before, so use with caution!

``gate_csv()`` now places all cells from all samples in the ``NBNode``. 

In [5]:
%%script false --no-raise-error

# This gate_csv takes about a minute
from nbnode.apply.gate_csv import gate_csv
all_files = [
        os.path.join(outdir, file_x)
        for file_x in os.listdir(outdir)
    ]
new_colnames = [
        "FS",
        "FS.0",
        "SS",
        "CD45RA",
        "CCR7",
        "CD28",
        "PD1",
        "CD27",
        "CD4",
        "CD8",
        "CD3",
        "CD57",
        "CD45",
    ]
celltree_gated = gate_csv(
        csv=all_files,
        celltree=complete_tree,
        new_colnames=new_colnames,
    )

example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 05 Tube 03 T cell 003 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 04 Tube 05 T cell 012 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter_Assay Donor 08 Tube 05 T cell 011 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 02 Tube 05 T cell 011 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 05 Tube 01 T cell 001 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 02 Tube 03 T cell 009 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter_Assay Donor 07 Tube 01 T cell 001 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 05 Tube 02 T cell 002 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter_Assay Donor 07 Tube 02 T cell 002 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 06 Tube 05 T cell 011 CD3.csv
example_data/asinh.align_manual.CD3_Gate/Inter Assay Donor 01 Tube 03 T cell 003 CD3.csv
example_data/asinh.al

In [6]:
%%script false --no-raise-error

celltree_gated.pretty_print()

AllCells (counter:1624608)
├── DN (counter:209833)
├── DP (counter:10245)
├── CD4-/CD8+ (counter:373102)
│   ├── naive (counter:161115)
│   │   ├── CD27+/CD28+ (counter:151533)
│   │   │   ├── CD57+/PD1+ (counter:1854)
│   │   │   ├── CD57+/PD1- (counter:182)
│   │   │   └── CD57- (counter:149497)
│   │   ├── CD27+/CD28- (counter:6282)
│   │   │   ├── CD57+/PD1+ (counter:2076)
│   │   │   ├── CD57+/PD1- (counter:1037)
│   │   │   └── CD57- (counter:3169)
│   │   ├── CD27-/CD28+ (counter:620)
│   │   │   ├── CD57+/PD1+ (counter:30)
│   │   │   ├── CD57+/PD1- (counter:6)
│   │   │   └── CD57- (counter:584)
│   │   └── CD27-/CD28- (counter:2680)
│   │       ├── CD57+/PD1+ (counter:1051)
│   │       ├── CD57+/PD1- (counter:1529)
│   │       └── CD57- (counter:100)
│   ├── Tcm (counter:49350)
│   │   ├── CD27+/CD28+ (counter:45707)
│   │   │   ├── CD57+/PD1+ (counter:2486)
│   │   │   ├── CD57+/PD1- (counter:174)
│   │   │   └── CD57- (counter:43047)
│   │   ├── CD27+/CD28- (counter:1889)
│

After we have the number of cells per node, we can generate a Dirichlet-based simulation. In particular, the number of cells per node are described by a single Dirichlet distribution containing a concentration parameter for every leaf node. 

In [7]:
%%script false --no-raise-error

from nbnode.simulation.FlowSimulationTree import FlowSimulationTreeDirichlet

flowsim_tree = FlowSimulationTreeDirichlet(
    rootnode=celltree_gated,
    include_features=new_colnames,
)

from nbnode.io.pickle_open_dump import pickle_open_dump

pickle_open_dump(flowsim_tree, "flowsim_tree.pickle")


/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/nbnode.py:364: UserWarning: data is no pandas.DataFrame, converting it via pd.DataFrame(data).
  warnings.warn(
/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/nbnode.py:353: UserWarning: self.ids was an empty list, subset an empty dataframe. Did you call celltree.id_preds(predicted_nodes)? Can also be a node with no cells.
  warnings.warn(
/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/FlowSimulationTree.py:485: UserWarning: Having zero percentages, because of dirichlet estimation replacing zero values with 2.079823630956095e-05
  warnings.warn(
/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/FlowSimulationTree.py:203: UserWarning: No cells in /AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1- to calculate any moments. Removing the population also from population_means and population_cov.
  warnings.warn(
/ho

In [27]:
import os 
import pickle
with open(os.path.join(os.pardir, os.pardir, "tests", "testdata", "flowcytometry", "flowsim_tree.pickle"), "rb") as f:
    flowsim_tree = pickle.load(f)

That's it. We can now simulate synthetic flow cytometry samples. 

In [30]:
flowsim_tree.sample(n_cells=10)

,FS,FS.0,SS,CD45RA,CCR7,CD28,PD1,CD27,CD4,CD8,CD3,CD57,CD45
0,0.102639,5.209169,0.079234,0.053654,0.657485,0.915508,0.993617,0.834220,0.970620,0.018259,0.170798,0.040396,0.243743
1,0.136999,5.295255,-0.485053,-0.029279,0.568698,0.868201,0.468776,0.969973,0.919147,0.004184,-0.649155,-0.050633,0.360048
2,0.246382,5.246198,0.340020,0.008805,-0.041882,1.067008,1.140333,0.049971,0.824414,0.066906,-0.761839,0.034460,0.266909
3,0.286864,5.247499,-0.107625,0.321744,-0.029727,0.741973,0.629558,0.061549,1.130998,0.114736,-0.230650,0.880764,0.364229
4,0.114729,5.148968,0.238837,0.581989,0.858279,1.028756,0.299741,0.834044,0.984013,0.054932,-0.123501,0.027360,-0.172184
5,-0.122504,5.209877,-0.112049,0.345653,0.983742,0.923229,0.316469,0.899396,0.904922,0.076794,0.361821,0.001648,0.133702
6,-0.053565,5.264089,-0.089595,0.356342,1.149574,0.946529,0.350388,1.060857,1.039667,0.011668,-0.334797,0.005728,-0.038644
7,0.031814,5.239712,-0.369597,0.694561,1.100095,1.052119,0.134514,1.064042,0.796746,0.053021,-0.152444,-0.041109,0.103083
8,-0.302928,5.235334,-0.389372,0.648083,1.302075,0.769360,0.487192,0.963519,0.022535,1.122902,-0.699979,0.001851,-0.234941
9,-0.306488,5.310085,-0.338834,0.952181,1.160882,0.656783,0.748290,0.902674,0.026744,1.025587,-0.161328,0.077204,-0.158110


In [31]:
synthetic_sample, cell_numbers = flowsim_tree.sample(n_cells=1000, return_sampled_cell_numbers=True)
print("Number of cells per population in the synthetic sample\n")
print(cell_numbers)

print("\n\n\nSynthetic sample")
synthetic_sample

Number of cells per population in the synthetic sample

/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+        5.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-        2.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-           238.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+        0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-             0.0
                                                    ...  
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+      0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-      0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-           0.0
/AllCells/DN                                        113.0
/AllCells/DP                                         13.0
Name: 0, Length: 96, dtype: float64



Synthetic sample


,FS,FS.0,SS,CD45RA,CCR7,CD28,PD1,CD27,CD4,CD8,CD3,CD57,CD45
0,-0.139501,5.359242,0.365108,0.061146,0.680750,1.298185,1.509557,0.723819,0.973619,0.034833,-0.262065,0.485445,0.109438
1,0.051763,5.258185,-0.259653,0.051309,0.165089,0.996564,1.230460,1.212181,1.011291,-0.046068,0.777399,0.704814,0.596709
2,0.170756,5.293380,0.096675,-0.024773,0.791767,0.877636,0.986675,0.872665,1.118470,0.023575,0.188112,0.433135,0.305276
3,0.206160,5.322153,-0.075823,0.075168,0.745799,1.003429,1.056580,0.919060,1.109823,0.025436,-0.398858,0.946519,-0.032292
4,-0.022061,5.110808,0.101293,-0.005731,0.602666,1.282100,0.935240,0.901593,1.091641,0.026834,-0.456788,0.631686,0.207932
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.070957,5.402411,-0.098018,0.279609,0.712786,1.251349,1.404615,0.517993,1.193985,0.516822,1.880004,0.686445,0.296492
996,0.482127,5.217043,0.650551,0.503816,0.914076,0.925214,0.033853,0.513948,0.523495,0.755007,-0.664136,-0.424347,-0.358827
997,0.054299,5.456519,0.796015,0.457269,0.786397,1.102934,0.964986,0.093859,0.587038,0.721107,-1.121974,0.988530,-0.678420
998,-0.030849,5.462171,0.143273,-0.126762,0.902751,1.487925,0.205410,0.807973,0.745973,0.618169,-0.769367,-0.085771,0.294419


If you do not need the actual cell ``features`` but only the number of cells per ``cell population``, you can do that with ``sample_populations()``

In [32]:
cell_numbers = flowsim_tree.sample_populations(n_cells=1000)
cell_numbers

/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+        0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-        0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-           261.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+        0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-             1.0
                                                    ...  
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+      2.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-      0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-           0.0
/AllCells/DN                                         85.0
/AllCells/DP                                          6.0
Name: 0, Length: 96, dtype: float64

Internally: 

 1. ``sample_populations()`` is called to generate the number of cells per leaf 
 2. ``multivariate_normal()`` generates the cell ``features`` for each cell

 You can also set a seed for reproducibility. 

In [33]:
flowsim_tree.set_seed(42)
synthetic_sample = flowsim_tree.sample(n_cells=3)
print(synthetic_sample)

flowsim_tree.set_seed(42)
synthetic_sample = flowsim_tree.sample(n_cells=3)
print(synthetic_sample)

         FS      FS.0        SS    CD45RA      CCR7      CD28       PD1   
0  0.033016  5.212589  0.027994 -0.042323  1.068687  1.230491  0.727421  \
1 -0.034233  5.219806  0.041332  0.009111  0.016646  1.218523  0.445606   
2  0.395116  5.366544  0.085398  0.737223  0.762656  0.641986  1.200716   

       CD27       CD4       CD8       CD3      CD57      CD45  
0  0.667649  1.017606 -0.012887 -0.232470 -0.015077  0.049724  
1  0.971163  1.167465  0.033722  0.081410  0.019712 -0.191248  
2  1.028306 -0.014907  0.892293 -0.686076 -0.011778  0.000069  
         FS      FS.0        SS    CD45RA      CCR7      CD28       PD1   
0  0.033016  5.212589  0.027994 -0.042323  1.068687  1.230491  0.727421  \
1 -0.034233  5.219806  0.041332  0.009111  0.016646  1.218523  0.445606   
2  0.395116  5.366544  0.085398  0.737223  0.762656  0.641986  1.200716   

       CD27       CD4       CD8       CD3      CD57      CD45  
0  0.667649  1.017606 -0.012887 -0.232470 -0.015077  0.049724  
1  0.971163  1

Per default, the normal distribution is based on a diagonal covariance matrix estimated on the cells per leaf node. However, you can enable the full covariance matrix!

In [34]:
flowsim_tree.set_seed(42)
# default
synthetic_sample = flowsim_tree.sample(n_cells=3, use_only_diagonal_covmat=True) 
print(synthetic_sample)

flowsim_tree.set_seed(42)
synthetic_sample = flowsim_tree.sample(n_cells=3, use_only_diagonal_covmat=False)
print(synthetic_sample)

         FS      FS.0        SS    CD45RA      CCR7      CD28       PD1   
0  0.033016  5.212589  0.027994 -0.042323  1.068687  1.230491  0.727421  \
1 -0.034233  5.219806  0.041332  0.009111  0.016646  1.218523  0.445606   
2  0.395116  5.366544  0.085398  0.737223  0.762656  0.641986  1.200716   

       CD27       CD4       CD8       CD3      CD57      CD45  
0  0.667649  1.017606 -0.012887 -0.232470 -0.015077  0.049724  
1  0.971163  1.167465  0.033722  0.081410  0.019712 -0.191248  
2  1.028306 -0.014907  0.892293 -0.686076 -0.011778  0.000069  
         FS      FS.0        SS    CD45RA      CCR7      CD28       PD1   
0 -0.014040  5.287691  0.113469 -0.056042  0.963664  1.316469  0.788263  \
1  0.077410  5.221344  0.198341 -0.028692  0.049742  1.251280  0.402444   
2 -0.093948  5.399214  0.514198  1.252213  1.269626  0.644032  0.538064   

       CD27       CD4       CD8       CD3      CD57      CD45  
0  1.189001  1.073660  0.097426 -0.041899  0.009045 -0.067505  
1  0.653476  0

### Synthetic data with effects

Previously, our ability was limited to mimicing the original dataset. However, we can also introduce effects in any ``cell population``! This means we can modify the proportion of cells derived from a particular ``cell population`` in a synthesized sample. 
Specifically, we defined the proportion of cells in a population using a Dirichlet distribution and its corresponding concentration parameters. By altering these parameters, we can adjust the number of cells in the population.

Note that we are not changing how the cells are samples _within_ a leaf node!



We show two ways to simulate different effects: 

1. ``TreeMeanRelative()``: Change any ``cell population`` by a relative amount. 
2. ``TreeMeanDistributionSampler()``: Force any distribution on a single ``cell population``

``TreeMeanRelative`` changes the concentration of a ``cell population`` by a certain factor (e.g. old = 173.2, new = old*2) and all other concentration parameters decrease (by a factor) such that the overall sum of concentration parameters remains equal. 

``TreeMeanDistributionSampler()`` first samples a _target_ proportion from a given distribution and then sets the concentration parameter such that on average the cell population contains this proportion. Again, the sum of concentration parameters remains equal.

#### ``TreeMeanRelative``

The following sets up a ``TreeMeanRelative`` where the cell population ``/AllCells/DN`` is changed by the factor 1 (therefore remains equal). 
``n_samples`` is set to 2 and ``n_cells`` to 100. 

Therefore, calling ``tmr.sample()`` generates two samples with the original distribution, 100 cells each. 
``tmr.sample()`` returns the number of cells per leaf population for both samples, the parameters after changing (the original tree is not changed) and the actually sampled samples. 

In [35]:
from nbnode.simulation.TreeMeanRelative import TreeMeanRelative
tmr = TreeMeanRelative(
    flowsim_tree,
    change_pop_mean_proportional={"/AllCells/DN": 1},
    n_samples=2,
    n_cells=100,
    verbose=False
)
print(tmr)
true_popcounts, changed_parameters, sampled_samples = tmr.sample()

true_popcounts

,sample_0,sample_1
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-,31.0,29.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-,1.0,0.0
...,...,...
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-,0.0,1.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-,0.0,0.0
/AllCells/DN,6.0,6.0


In [36]:
changed_parameters["alpha"]  # these are the concentration parameters of the Dirichlet distribution

/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+       0.901580
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-       0.295210
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-           48.203633
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+       0.091623
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-            0.096534
                                                      ...    
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+     0.159252
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-     0.153660
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-          0.125163
/AllCells/DN                                        15.218566
/AllCells/DP                                         1.755718
Name: 0, Length: 96, dtype: float64

In [37]:
print(f"There are {len(sampled_samples)} synthetic samples, showing only the first\n")
sampled_samples[0]

There are 2 synthetic samples, showing only the first



,FS,FS.0,SS,CD45RA,CCR7,CD28,PD1,CD27,CD4,CD8,CD3,CD57,CD45
0,0.062235,5.241828,0.167840,-0.084488,0.462969,1.327377,1.020327,0.735205,0.919547,-0.024936,-0.459670,-0.024190,-0.155744
1,-0.137097,5.211038,0.006071,-0.013674,0.631770,1.354644,0.924415,1.071354,0.909430,-0.011694,-0.594609,-0.015247,-0.368281
2,0.049543,5.226160,-0.092377,0.032866,0.720488,1.218761,0.788295,1.015296,1.006380,0.065456,-0.138280,0.082154,0.100491
3,-0.161587,5.303274,-0.203429,-0.063234,0.272455,1.249179,0.712787,1.112724,1.045490,0.031186,-0.482921,0.040894,-0.401152
4,0.153456,5.171440,0.188800,0.084201,0.742456,1.389247,0.397038,0.904283,1.021047,0.004681,-0.283981,-0.004825,-0.402051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.315077,5.217177,0.494209,-0.770656,-0.407820,0.650297,1.125016,0.144094,-0.011654,0.032542,0.723826,1.004760,0.156519
96,0.534101,5.270793,0.460711,0.347915,0.157224,0.784490,1.131765,0.869825,0.019530,-0.018556,0.217103,0.367984,0.458230
97,-0.226989,5.252685,-0.189814,0.154086,-0.156006,0.536826,1.165235,0.703251,0.030829,-0.052641,0.138474,0.634439,0.154382
98,-0.001238,5.322612,0.424035,0.193686,0.277735,1.124228,0.598610,0.624192,0.010864,-0.030898,0.231010,-0.259944,0.003429


Increasing a specific proportion on average increases the respective cell population's number of cells: 

In [38]:
tmr_base = TreeMeanRelative(
    flowsim_tree,
    change_pop_mean_proportional={"/AllCells/DN": 1},
    n_samples=10,
    n_cells=100,
    verbose=False
)
tmr_ridiculously_changed = TreeMeanRelative(
    flowsim_tree,
    change_pop_mean_proportional={"/AllCells/DN": 10},
    n_samples=10,
    n_cells=100,
    verbose=False
)


You can check the changed parameters when sampling from the distribution: 

In [39]:
true_popcounts, changed_parameters, sampled_samples = tmr_base.sample()
r_true_popcounts, r_changed_parameters, r_sampled_samples = tmr_ridiculously_changed.sample()

changed_parameters["alpha"]

/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+       0.901580
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-       0.295210
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-           48.203633
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+       0.091623
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-            0.096534
                                                      ...    
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+     0.159252
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-     0.153660
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-          0.125163
/AllCells/DN                                        15.218566
/AllCells/DP                                         1.755718
Name: 0, Length: 96, dtype: float64

In [40]:
r_changed_parameters["alpha"]

/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+        0.169073
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-        0.055361
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-             9.039610
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+        0.017182
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-             0.018103
                                                       ...    
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+      0.029865
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-      0.028816
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-           0.023472
/AllCells/DN                                        152.185662
/AllCells/DP                                          0.329249
Name: 0, Length: 96, dtype: float64

We observe that the concentration parameter of ``/AllCells/DN`` strongly increased! Therefore also the number of cells from that population changed

In [41]:
true_popcounts

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-,31.0,29.0,36.0,31.0,22.0,30.0,29.0,24.0,29.0,31.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/DN,6.0,6.0,7.0,7.0,17.0,9.0,8.0,13.0,12.0,5.0


In [42]:
r_true_popcounts

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-,4.0,4.0,6.0,4.0,3.0,4.0,5.0,3.0,6.0,5.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/AllCells/DN,86.0,90.0,90.0,90.0,83.0,85.0,82.0,89.0,85.0,85.0


#### ``TreeMeanDistributionSampler``

The following sets up a ``TreeMeanDistributionSampler`` where the cell population ``/AllCells/DN`` is sampled from a normal distribution with mean of the original data and a standard deviation of 1 (default).  
``n_samples`` is set to 2 and ``n_cells`` to 100. 

Therefore, calling ``tmr.sample()`` generates two samples with the values from a normal distribution with a mean of 8.28% being in that population, a standard deviation of 1. The value from that normal distribution is then set as target for the Dirichlet distribution, and 100 cells are synthesized. 


``tmr.sample()`` returns the number of cells per leaf population for both samples, the parameters after changing (the original tree is not changed) and the actually sampled samples. 

In [43]:
from nbnode.simulation.TreeMeanDistributionSampler import TreeMeanDistributionSampler

tmds = TreeMeanDistributionSampler(
    flowsim_tree,
    population_name_to_change="/AllCells/DN",
    n_samples=2,
    n_cells=100,
    verbose=False,
)
(
    all_true_popcounts,
    all_changed_parameters,
    all_sampled_samples,
    all_targets,
) = tmds.sample()



print(flowsim_tree.pop_mean("/AllCells/DN"))
all_true_popcounts

0.08279977774316377


/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/sim_target.py:90: UserWarning: Changing more than 1 population is NOT setting the means to the specifiedvalues - only the LAST value will be exactly the set percentage
  warnings.warn(


,sample_0,sample_1
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-,32.0,29.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+,0.0,0.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-,0.0,0.0
...,...,...
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-,0.0,1.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-,0.0,0.0
/AllCells/DN,6.0,8.0


Alternatively, you can insert any population distribution with a function ``mean_distribution`` taking one argument (``original_mean``) and returning a class with a ``.sample()`` method which can be casted to ``float``. 

Internally, the ``target_mean_distribution`` is set to ``mean_distribution(original_mean)`` of the original population. 
Then for every of the ``n_samples``, a single value is sampled by ``float(target_mean_distribution.sample())`` and this value is set as mean of the Dirichlet distribution for the ``population_name_to_change``. 

We included a ``PseudoTorchDistributionNormal`` with the relevant method, but you can use any distribution from [pytorch-distributions](https://pytorch.org/docs/stable/distributions.html). 

In the following example, we introduce a much higher variance on one population by basing it on a normal distribution with scale 10. 

In [47]:
from nbnode.simulation.TreeMeanDistributionSampler import PseudoTorchDistributionNormal

tmds_custom = TreeMeanDistributionSampler(
    flowsim_tree,
    population_name_to_change="/AllCells/DN",
    mean_distribution=lambda original_mean: PseudoTorchDistributionNormal(
        loc=original_mean, scale=10
    ),
    n_samples=10,
    n_cells=10000,
    verbose=False,
)
(
    all_true_popcounts,
    all_changed_parameters,
    all_sampled_samples,
    all_targets,
) = tmds_custom.sample()

all_true_popcounts


/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/sim_target.py:90: UserWarning: Changing more than 1 population is NOT setting the means to the specifiedvalues - only the LAST value will be exactly the set percentage
  warnings.warn(
/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/sim_target.py:90: UserWarning: Changing more than 1 population is NOT setting the means to the specifiedvalues - only the LAST value will be exactly the set percentage
  warnings.warn(
/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/sim_target.py:90: UserWarning: Changing more than 1 population is NOT setting the means to the specifiedvalues - only the LAST value will be exactly the set percentage
  warnings.warn(
/home/gugl/.conda_envs/nbnode_pyscaffold/lib/python3.8/site-packages/nbnode/simulation/sim_target.py:90: UserWarning: Changing more than 1 population is NOT setting the means to the sp

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1+,23.0,40.0,50.0,125.0,23.0,81.0,32.0,50.0,29.0,81.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57+/PD1-,0.0,0.0,0.0,0.0,22.0,0.0,0.0,23.0,1.0,4.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28+/CD57-,2671.0,2698.0,2865.0,2693.0,2262.0,2362.0,3006.0,2262.0,2621.0,2837.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57+/PD1+,0.0,32.0,32.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0
/AllCells/CD4+/CD8-/Tcm/CD27+/CD28-/CD57-,27.0,0.0,0.0,0.0,0.0,4.0,14.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1+,0.0,13.0,0.0,0.0,3.0,14.0,0.0,9.0,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57+/PD1-,0.0,165.0,0.0,4.0,24.0,6.0,1.0,0.0,0.0,0.0
/AllCells/CD4-/CD8+/naive/CD27-/CD28-/CD57-,44.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
/AllCells/DN,529.0,111.0,2215.0,1092.0,524.0,1056.0,0.0,1677.0,954.0,497.0
